# Day 3: Gear Ratios

The engineer explains that an engine part seems to be missing from the engine, but nobody can figure out which one. If you can add up all the part numbers in the engine schematic, it should be easy to work out which part is missing.

The engine schematic (your puzzle input) consists of a visual representation of the engine. There are lots of numbers and symbols you don't really understand, but apparently any number adjacent to a symbol, even diagonally, is a "part number" and should be included in your sum. (Periods (.) do not count as a symbol.)

## Part 1

In [3]:
module Data =
    let load fileName =
        File.ReadLines fileName

module Combinators =
    let C a b c = a c b
    let K a _ = a
    let Φ a b c d = a (b d) (c d)
        
module Matrix =
    let private emptyRow defaultValue =
        Seq.map (Combinators.K defaultValue)

    let shiftUp defaultValue matrix =
        matrix
        |> Seq.skip 1
        |> Combinators.C Seq.append (Seq.singleton (emptyRow defaultValue (Seq.head matrix)))
        
    let shiftDown defaultValue matrix =
        matrix
        |> Seq.take (Seq.length matrix - 1)
        |> Seq.append (Seq.singleton (emptyRow defaultValue (Seq.head matrix)))

module Schematic =

    let private maskCharacter (character: char) =
        match character with
        | '.' -> 0
        | x when Char.IsDigit(x) -> 0
        | _ -> 1
        

    let private maskSymbols line =
        line |> Seq.map maskCharacter

    let combineMasks (a, b, c) =
        a ||| b ||| c

    let mask schematic =
        let mask = schematic |> Seq.map maskSymbols
        mask
        |> Combinators.Φ (Seq.zip3 mask) (Matrix.shiftUp 0) (Matrix.shiftDown 0)
        |> Seq.map ((<|||) Seq.zip3)
        |> Seq.map (Seq.map combineMasks)

    let private add xs x =
        xs @ [x]
        
    let private setKeep (results, keep) (take:int, character: char) =
        match take, character with
        | 0, '.' -> (add results (take, character), 0)
        | _ -> (add results (keep ||| take, character), keep ||| take)
        
    let private remove (take: int, character: char) =
        match take with
        | 0 -> (0, '.')
        | 1 when Char.IsDigit(character) |> not -> (0, '.')
        | _ -> (take, character)

    let private unmaskNumbers (results, accumulate) (character: char) =
        match character with
        | '.' when List.isEmpty accumulate |> not -> (add results accumulate, List.empty)
        | '.' -> (results, accumulate)
        | _ -> (results, add accumulate character)
        
    let findPartNumbers schematic =
        schematic
        |> Seq.zip (mask schematic)
        |> Seq.map ((<||) Seq.zip)
        |> Seq.reduce Seq.append
        |> Seq.toList
        |> List.fold setKeep (List.empty, 0)
        |> fst
        |> List.rev
        |> List.fold setKeep (List.empty, 0)
        |> fst
        |> List.rev
        |> List.map (remove >> snd)
        |> List.fold unmaskNumbers (List.empty, List.empty)
        |> fst
        |> List.map (Seq.toArray >> String >> int)



### Test

In [4]:
"Day 3 - Part 1 - Test.txt"
|> Data.load
|> Schematic.findPartNumbers
|> Seq.sum

4361

### Solution

In [5]:
"Day 3 - Part 1.txt"
|> Data.load
|> Schematic.findPartNumbers
|> Seq.sum

528799

## Part 2

The missing part wasn't the only issue - one of the gears in the engine is wrong. A gear is any * symbol that is adjacent to exactly two part numbers. Its gear ratio is the result of multiplying those two numbers together.

This time, you need to find the gear ratio of every gear and add them all up so that the engineer can figure out which gear needs to be replaced.

In [26]:
module Gears =
    let addCoordinates row columns =
        columns
        |> Seq.mapi (fun column item -> (row, column, item))

    let find schematic =
        schematic
        |> Seq.mapi addCoordinates
        

    

### Test

In [27]:
"Day 3 - Part 1 - Test.txt"
|> Data.load
|> Seq.map (Seq.map id)
|> Gears.find


[ [ (0, 0, 4), (0, 1, 6), (0, 2, 7), (0, 3, .), (0, 4, .), (0, 5, 1), (0, 6, 1), (0, 7, 4), (0, 8, .), (0, 9, .) ], [ (1, 0, .), (1, 1, .), (1, 2, .), (1, 3, *), (1, 4, .), (1, 5, .), (1, 6, .), (1, 7, .), (1, 8, .), (1, 9, .) ], [ (2, 0, .), (2, 1, .), (2, 2, 3), (2, 3, 5), (2, 4, .), (2, 5, .), (2, 6, 6), (2, 7, 3), (2, 8, 3), (2, 9, .) ], [ (3, 0, .), (3, 1, .), (3, 2, .), (3, 3, .), (3, 4, .), (3, 5, .), (3, 6, #), (3, 7, .), (3, 8, .), (3, 9, .) ], [ (4, 0, 6), (4, 1, 1), (4, 2, 7), (4, 3, *), (4, 4, .), (4, 5, .), (4, 6, .), (4, 7, .), (4, 8, .), (4, 9, .) ], [ (5, 0, .), (5, 1, .), (5, 2, .), (5, 3, .), (5, 4, .), (5, 5, +), (5, 6, .), (5, 7, 5), (5, 8, 8), (5, 9, .) ], [ (6, 0, .), (6, 1, .), (6, 2, 5), (6, 3, 9), (6, 4, 2), (6, 5, .), (6, 6, .), (6, 7, .), (6, 8, .), (6, 9, .) ], [ (7, 0, .), (7, 1, .), (7, 2, .), (7, 3, .), (7, 4, .), (7, 5, .), (7, 6, 7), (7, 7, 5), (7, 8, 5), (7, 9, .) ], [ (8, 0, .), (8, 1, .), (8, 2, .), (8, 3, $), (8, 4, .), (8, 5, *), (8, 6, .), (8, 7, .), (8, 8, .), (8, 9, .) ], [ (9, 0, .), (9, 1, 6), (9, 2, 6), (9, 3, 4), (9, 4, .), (9, 5, 5), (9, 6, 9), (9, 7, 8), (9, 8, .), (9, 9, .) ] ] f Microsoft.FSharp.Collections.SeqModule+revamp@728[System.Collections.Generic.IEnumerable`1[System.Char],System.Collections.Generic.IEnumerable`1[System.Tuple`3[System.Int32,System.Int32,System.Char]]] f Microsoft.FSharp.Collections.SeqModule+MapIndexed@753-1[System.Collections.Generic.IEnumerable`1[System.Char],System.Collections.Generic.IEnumerable`1[System.Tuple`3[System.Int32,System.Int32,System.Char]]] mapping FSI_0040+Gears+find@10-2[System.Collections.Generic.IEnumerable`1[System.Char],System.Char] ie [ [ 4, 6, 7, ., ., 1, 1, 4, ., . ], [ ., ., ., *, ., ., ., ., ., . ], [ ., ., 3, 5, ., ., 6, 3, 3, . ], [ ., ., ., ., ., ., #, ., ., . ], [ 6, 1, 7, *, ., ., ., ., ., . ], [ ., ., ., ., ., +, ., 5, 8, . ], [ ., ., 5, 9, 2, ., ., ., ., . ], [ ., ., ., ., ., ., 7, 5, 5, . ], [ ., ., ., $, ., *, ., ., ., . ], [ ., 6, 6, 4, ., 5, 9, 8, ., . ] ] f Microsoft.FSharp.Collections.SeqModule+revamp@728[System.String,System.Collections.Generic.IEnumerable`1[System.Char]] f Microsoft.FSharp.Collections.SeqModule+Map@748-1[System.String,System.Collections.Generic.IEnumerable`1[System.Char]] mapping FSI_0041+it@3-18 ie [ 467..114.., ...*......, ..35..633., ......#..., 617*......, .....+.58., ..592....., ......755., ...$.*...., .664.598.. ] Current <null> (values) [ 467..114.., ...*......, ..35..633., ......#..., 617*......, .....+.58., ..592....., ......755., ...$.*...., .664.598.. ] (values) index value 0 [ 4, 6, 7, ., ., 1, 1, 4, ., . ] f Microsoft.FSharp.Collections.SeqModule+revamp@728[System.Char,System.Char] f Microsoft.FSharp.Collections.SeqModule+Map@748-1[System.Char,System.Char] mapping FSI_0041+it@3-19 ie 467..114.. (values) [ 4, 6, 7, ., ., 1, 1, 4, ., . ] 1 [ ., ., ., *, ., ., ., ., ., . ] f Microsoft.FSharp.Collections.SeqModule+revamp@728[System.Char,System.Char] f Microsoft.FSharp.Collections.SeqModule+Map@748-1[System.Char,System.Char] mapping FSI_0041+it@3-19 ie ...*...... (values) [ ., ., ., *, ., ., ., ., ., . ] 2 [ ., ., 3, 5, ., ., 6, 3, 3, . ] f Microsoft.FSharp.Collections.SeqModule+revamp@728[System.Char,System.Char] f Microsoft.FSharp.Collections.SeqModule+Map@748-1[System.Char,System.Char] mapping FSI_0041+it@3-19 ie ..35..633. (values) [ ., ., 3, 5, ., ., 6, 3, 3, . ] 3 [ ., ., ., ., ., ., #, ., ., . ] f Microsoft.FSharp.Collections.SeqModule+revamp@728[System.Char,System.Char] f Microsoft.FSharp.Collections.SeqModule+Map@748-1[System.Char,System.Char] mapping FSI_0041+it@3-19 ie ......#... (values) [ ., ., ., ., ., ., #, ., ., . ] 4 [ 6, 1, 7, *, ., ., ., ., ., . ] f Microsoft.FSharp.Collections.SeqModule+revamp@728[System.Char,System.Char] f Microsoft.FSharp.Collections.SeqModule+Map@748-1[System.Char,System.Char] mapping FSI_0041+it@3-19 ie 617*...... (values) [ 6, 1, 7, *, ., ., ., ., ., . ] 5 [ ., ., ., ., ., +, ., 5, 8, . ] f Microsoft.FSharp.Collections.SeqModule+revamp@728[System.